<a href="https://colab.research.google.com/github/stswee/IntroCompStatsHSSP2023/blob/main/Class_Code/Intro_to_Comp_Statistics_Day_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Computational Statistics (HSSP 2023 Edition)
## Day 3: Monte Carlo and Bootstrapping

In this notebook, we will run exact statistical tests, Monte Carlo simulations, and bootstrapping programs.

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2
import scipy.stats as stats
import random
import math

### Activity 1: Rolling a Fair Dice

Problem Statement: You want to see if a dice is fair or not. After 60 rolls, you record:


*   1: 10
*   2: 3
*   3: 12
*   4: 24
*   5: 6
*   6: 5

Compute the exact and estimated (monte-carlo) p-value.



In [ ]:
# By hand calculations
observed = np.array([10, 3, 12, 24, 6, 5])
expected = np.array([10, 10, 10, 10, 10, 10])

# Calculate test statistic
test_statistic = 0
for i in range(len(observed)):
  test_statistic += (observed[i] - expected[i])**2 / expected[i]
test_statistic = int(test_statistic) # Exact value is 29, so remove floating-point approximation

# Calculate degrees of freedom
df = len(observed) - 1

# Determine p-value
# Remember that we want the area to the right
pval = 1 - chi2.cdf(test_statistic, df, loc=0, scale=1)

print('Test statistic is : ' + str(test_statistic))
print('p value : ' + str(pval))

In [ ]:
# Using stats package
test_statistic, pval = stats.chisquare(observed, expected)
print('Test statistic is : ' + str(test_statistic))
print('p value : ' + str(pval))

In [ ]:
# Monte Carlo Simulation
# Parameters
B = 100000 # Number of samples (for B = 100000, about 5-10 seconds using by hand code; about 30 seconds with package code)
low = 1
high = 6
expected = np.array([10, 10, 10, 10, 10, 10])

# Storage for array of test statistics
mc_test_statistics = np.empty(B)

# Run Simulation
for i in range(B):
  # Generate random sample
  mc_sample = [random.randint(low, high) for j in range(60)]

  # Determine counts of each integer
  mc_sample = [mc_sample.count(i) for i in range(1, 7)]

  # By hand code
  # Calculate test statistic
  # mc_test_statistic = 0
  # for j in range(len(mc_sample)):
  #   mc_test_statistic += (mc_sample[j] - expected[j])**2 / expected[j]

  # Package code
  mc_test_statistic = stats.chisquare(mc_sample, expected)[0]

  # Store test statistic
  mc_test_statistics[i] = mc_test_statistic

# MC p-value
pval = (sum(mc_test_statistics > test_statistic) + 1)/ (B + 1)
print(pval)

### Activity 2: Estimating pi

Problem Statement:

Estimate pi using a Monte Carlo simulation

In [ ]:
# Run Monte Carlo Simulation (no plot)
num_points = 10000

# Keep track of number of points
circle_points = 0
square_points = 0

# Generate points
for i in range(num_points):
  # Generate random x and y values
  x = random.uniform(0, 1)
  y = random.uniform(0, 1)

  # Determine distance from origin
  d = math.sqrt(x**2 + y**2)

  # If distance is less than 1, then the point is inside the quarter-circle
  if (d <= 1):
    circle_points += 1

  # Increase number of square points
  square_points += 1

  # Generate estimate for pi every 1000 points
  if ((i % 1000) == 0):
    pi = round(4 * circle_points / square_points, 4)
    error = round(math.pi - pi, 4)
    print("The estimate of pi is " + str(float(pi)) + " with an error of " + str(float(error)))


In [ ]:
# Run Monte Carlo Simulation (with plot)
num_points = 100000

# Keep track of number of points
circle_points = 0
square_points = 0

# Store points
circle_storage = np.empty([0, 2])
square_storage = np.empty([0, 2])

# Generate points
for i in range(num_points):
  # Generate random x and y values
  x = random.uniform(0, 1)
  y = random.uniform(0, 1)

  # Determine distance from origin
  d = math.sqrt(x**2 + y**2)

  # If distance is less than 1, then the point is inside the quarter-circle
  if (d <= 1):
    circle_points += 1
    circle_storage = np.append(circle_storage, [[x, y]], axis = 0)
  else:
    square_storage = np.append(square_storage, [[x, y]], axis = 0)

  # Increase number of square points
  square_points += 1

  # Generate a plot for pi every 10000 points
  if ((i % 10000) == 0):
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    ax1.scatter(circle_storage[:,0], circle_storage[:,1], s=5, c='r', marker="o", label='circle')
    ax1.scatter(square_storage[:,0], square_storage[:,1], s=5, c='b', marker="o", label='square')
    plt.legend(loc='upper left')
    plt.show()


### Activity 3: Rolling 2 Dice

Problem Statement:

You roll two four-sided dice and you want to see if there is a correlation between the rolls. You gather the following data:

In [ ]:
data = np.array([[15, 28, 30, 12], [22, 41, 10, 8], [30, 42, 28, 19], [31, 20, 10, 10]])
print(data)

Each element can be thought of as a coordinate. For example, the 12 on the top right is (1, 4). This reflects the number of times we rolled 1 on the first dice and 4 on the second.

Compute the exact and estimated (bootstrap) p-value.

In [ ]:
# By hand calculations
# Initialize totals
row_total = np.zeros(data.shape[0])
column_total = np.zeros(data.shape[1])
sample_size = sum(sum(data))

for i in range(len(row_total)):
  row_total[i] = sum(data[i,:])

for j in range(len(column_total)):
  column_total[j] = sum(data[:, j])

# Alternatively, for column total, can use the code below
# column_total = sum(data)

# Determine expected counts array
expected_counts = np.zeros(data.shape)

for i in range(len(row_total)):
  for j in range(len(column_total)):
    expected_counts[i, j] = row_total[i] * column_total[j] / sample_size

# Calculate chi-square test statistic
test_statistic = 0
for i in range(data.shape[0]):
  for j in range(data.shape[1]):
    test_statistic += (data[i, j] - expected_counts[i,j])**2 / (expected_counts[i, j])

# Determine degrees of freedom
df = (data.shape[0] - 1) * (data.shape[1] - 1)

# Determine p-value
pval = 1 - chi2.cdf(test_statistic, df, loc=0, scale=1)

print('Test statistic is : ' + str(test_statistic))
print('p value : ' + str(pval))

In [ ]:
# Using stats package
test_statistic, pval = stats.chi2_contingency(data)[0:2]
print('Test statistic is : ' + str(test_statistic))
print('p value : ' + str(pval))

In [ ]:
# Bootstrap
# Initialize totals
row_total = np.zeros(data.shape[0])
column_total = np.zeros(data.shape[1])
sample_size = sum(sum(data))

for i in range(len(row_total)):
  row_total[i] = sum(data[i,:])

for j in range(len(column_total)):
  column_total[j] = sum(data[:, j])

# Estimate null distribution
row_dist = row_total / sum(row_total)
column_dist = column_total / sum(column_total)

In [ ]:
# Function definition to draw from distribution
def draw_integers(integers, probabilities, n):
    random_integers = random.choices(integers, probabilities, k = n)
    return random_integers

In [ ]:
# Run bootstrap
B = 10000 # Number of samples (for B = 10000, about 90 seconds)

# Storage for array of test statistics
boot_test_statistics = np.empty(B)

for b in range(B):
  # Draw from estimated null distribution
  x = np.array(draw_integers([1, 2, 3, 4], row_dist, sample_size))
  y = np.array(draw_integers([1, 2, 3, 4], column_dist, sample_size))

  # Get contingency table
  boot_contingency_table = np.histogram2d(x, y, bins=[np.max(x), np.max(y)])[0]

  # Calculate test statistic
  boot_test_statistic = stats.chi2_contingency(boot_contingency_table)[0]

  # Store test statistic
  boot_test_statistics[i] = boot_test_statistic

# Bootstrap p-value
pval = (sum(boot_test_statistics > test_statistic) + 1)/ (B + 1)
print(pval)